In [17]:
#loading the required packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import plotnine as p9 
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
#Fetching the data
raw_data = pd.read_csv('./data/tesco-dataset/train.csv')
print('The shape of the dataset :' + str(raw_data.shape))
raw_data.head()

The shape of the dataset :(320, 16)


,location_id,crime_rate,proportion_flats,proportion_nonretail,new_store,commercial_property,household_size,proportion_newbuilds,public_transport_dist,transport_availability,property_value,school_proximity,competitor_density,household_affluency,normalised_sales,county
0,464,17.600541,0.0,18.10,no,NaN,2.926,29.0,2.9084,All transport options,666,20.2,368.74,4.5325,-0.399933,c_40
1,504,0.603556,20.0,3.97,no,14.85,4.520,10.6,2.1398,Average transport options,264,13.0,388.37,1.8150,2.216308,c_80
2,295,0.606810,0.0,6.20,no,7.70,2.981,31.9,3.6715,Many transport options,307,17.4,378.35,2.9125,0.166920,c_53
3,187,0.012385,55.0,2.25,no,1.95,3.453,68.1,7.3073,No transport options,300,15.3,394.72,2.0575,-0.083804,c_65
4,193,0.016182,100.0,1.32,no,3.05,3.816,59.5,8.3248,Average transport options,256,15.1,392.90,0.9875,0.962693,c_97


# Plan for modelling (MVP):
* Usuals - > Train_test split, target variable split, 
1. Missing values for school_proximity and commercial_property - Start with median imputation and check the model accuracy
2. Categorical encoding
    1. Binary encoding for new_store feature
    2. Ordinal encoding for transport_availability feature
    3. Group counties to high, medium, low sales
3. Outlier handling if requred - not for MVP
4. Feature selection
5. Model development 

In [ ]:
# train_test split


In [15]:
# Ordinal transformation
order=[['No transport options','Few transport options','Average transport options','Many transport options','All transport options'        
        ]]
ordinal_encoder = OrdinalEncoder(categories=order)
transport_availability_encoded= ordinal_encoder.fit_transform(raw_data[["transport_availability"]])
ordinal_encoder.categories_

In [22]:
# onehot transformation
cat_encoder = OneHotEncoder(drop='if_binary')
new_store_encoded= cat_encoder.fit_transform(raw_data[["new_store"]])
cat_encoder.categories_

[array(['no', 'yes'], dtype=object)]

In [ ]:
# Finish the pipeline
